# Tic-Tac-Toe with Dependence Inversion, Open-Close and Liskov Substitution Principles

---

## Board
#### The abstract Board class allows for potential future extensions to different types of boards, which adheres to the OCP
#### Board's responsibilities are clear (managing the state and rules of the board itself), and delegate game-specific logic to another class. 
#### This would be in line with the Single Responsibility Principle (SRP).

In [209]:
from abc import ABC, abstractmethod

class Board(ABC):
    def __init__(self, boardDimensions):
        self.boardDimensions = boardDimensions

    @abstractmethod
    def createBoard(self):
        pass

    @abstractmethod
    def printBoard(self):
        pass

    @abstractmethod
    def setCellState(self, position, state):
        pass

    @abstractmethod
    def getCellState(self, position):
        pass

    @abstractmethod
    def getBoardState(self):
        pass

    def getBoardDimensions(self):
        pass

In [210]:
class TicTacToeBoard(Board):
    def __init__(self, boardDimensions):
        super().__init__(boardDimensions)
        self.createBoard()

    def createBoard(self):
        self.boardState = {i+1: ' ' for i in range(self.getBoardDimensions()**2)}

    def printBoard(self):
        for i in range(self.boardDimensions):
            row = [self.boardState[i*self.boardDimensions+j+1] for j in range(self.boardDimensions)]
            print('|'.join(row))
            if i < self.boardDimensions-1:
                print('-'*(self.boardDimensions*2-1))
        print('\n')

    def setCellState(self, position, state):
        self.boardState[position] = state

    def getCellState(self, position):
        return self.boardState[position]

    def getBoardState(self):
        return self.boardState

    def getBoardDimensions(self):
        return self.boardDimensions

---

## Game Logic
#### Abstract class handles the game-specific logic. 
#### Placed at the same level as Board, Player, and Algorithm (directly under GameLoop).
#### This is in line with the Single Responsibility Principle (SRP).

In [211]:
from abc import ABC, abstractmethod

class GameLogic():
    def __init__(self, boardGame):
        self.boardGame = boardGame

    @abstractmethod
    def chkForkWin(self):
        pass

    @abstractmethod
    def chkForDraw(self):
        pass

    @abstractmethod
    def chkMarkForWin(self, letter):
        pass

    @abstractmethod
    def insertLetter(self, letter, position):
        pass

    @abstractmethod
    def spaceIsFree(self, position):
        pass

In [212]:
class TicTacToeGameLogic(GameLogic):
    def __init__(self, boardGame):
        super().__init__(boardGame)

    # function to check for draw
    # check if all the spaces are taken
    def chkForDraw(self):
        boardState = self.boardGame.getBoardState()
        for key in boardState.keys():
            if boardState[key] == ' ':
                return False
        return True

    # function to check for win
    # check if any of the rows have all the same values (and not empty)
    # check if any of the columns have all the same values (and not empty)
    # check if any of the diagonals have all the same values (and not empty)
    def chkForWin(self):
        boardState = self.boardGame.getBoardState()
        boardDimensions = self.boardGame.getBoardDimensions()
        # check rows
        for i in range(boardDimensions):
            row = [boardState[i*boardDimensions+j+1] for j in range(boardDimensions)]
            if len(set(row)) == 1 and row[0] != ' ':
                return True
        # check columns
        for i in range(boardDimensions):
            column = [boardState[j*boardDimensions+i+1] for j in range(boardDimensions)]
            if len(set(column)) == 1 and column[0] != ' ':
                return True
        # check diagonals
        diagonal1 = [boardState[i*boardDimensions+i+1] for i in range(boardDimensions)]
        diagonal2 = [boardState[i*boardDimensions+(boardDimensions-i-1)+1] for i in range(boardDimensions)]
        if len(set(diagonal1)) == 1 and diagonal1[0] != ' ':
            return True
        if len(set(diagonal2)) == 1 and diagonal2[0] != ' ':
            return True
        return False

    # function to check which letter has won
    # return the letter that has won
    def chkMarkForWin(self, letter):
        boardState = self.boardGame.getBoardState()
        boardDimensions = self.boardGame.getBoardDimensions()
        # check rows
        for i in range(boardDimensions):
            row = [boardState[i*boardDimensions+j+1] for j in range(boardDimensions)]
            if len(set(row)) == 1 and row[0] == letter:
                return True
        # check columns
        for i in range(boardDimensions):
            column = [boardState[j*boardDimensions+i+1] for j in range(boardDimensions)]
            if len(set(column)) == 1 and column[0] == letter:
                return True
        # check diagonals
        diagonal1 = [boardState[i*boardDimensions+i+1] for i in range(boardDimensions)]
        diagonal2 = [boardState[i*boardDimensions+(boardDimensions-i-1)+1] for i in range(boardDimensions)]
        if len(set(diagonal1)) == 1 and diagonal1[0] == letter:
            return True
        if len(set(diagonal2)) == 1 and diagonal2[0] == letter:
            return True
        return False


    # insert letter function
    # check to see if space is free
    # if space is free, insert letter
    # if space is not free, ask user to pick a different position
    def insertLetter(self, letter, position):
        if (self.spaceIsFree(position)):
            self.boardGame.setCellState(position, letter)
            self.boardGame.printBoard()
        else:
            print("Can't insert there!")
            # ask user to pick a different position
            position = int(input("Please enter a different position: "))
            self.insertLetter(letter, position)

    # check if space is free
    def spaceIsFree(self, position):
        cellState = self.boardGame.getCellState(position)
        if cellState == ' ':
            return True
        else:
            return False

---

## Player
#### Abstract Player class.
#### Example of the OCP. Each type of Player can have its unique functionalities defined in its specific subclass. 
#### LSP is achieved by having the subclasses be substitutable for their base class.

In [213]:
from abc import ABC, abstractmethod

# an abstract class for a player
class Player(ABC):
    def __init__(self, letter, algorithm):
        self.letter = letter
        self.algorithm = algorithm
    
    @abstractmethod
    # function for player to choose an algorithm
    def chooseAlgorithm(self):
        pass

    @abstractmethod
    # function for player to choose a position
    def makeMove(self):
        pass

In [214]:
# create a player class
class Player:
    def __init__(self, letter, algorithm):
        self.letter = letter
        self.algorithm = algorithm

    def chooseAlgorithm(self, algorithm):
        if algorithm == 1:
            self.algorithm = Minimax()
        elif algorithm == 2:
            self.algorithm = ReinforcementLearning()
        else:
            self.algorithm = UserInput()

    def makeMove(self, board):
        return self.algorithm.bestMove(board, self.letter)

---

### Algorithms
#### The abstract Algorithm class, used by the ComputerPlayer and HumanPlayer.
#### Allows for different AI strategies to be easily interchanged and potentially added in the future, following the OCP.

In [215]:
# an abstract class for an algorithm
# it has a function called bestMove that returns the best move
class Algorithm(ABC):
    def __init__(self, boardGame):
        self.boardGame = boardGame
    
    @abstractmethod
    def bestMove(self, boardGame, letter):
        pass


#### Minimax plays suboptimal moves when the opponent plays suboptimally, but plays optimally when the opponent plays optimally.
#### There is an option to create a list of best moves. Then randomly choose a move from the list.
#### Not sure how much of a difference this would make, but it would be interesting to test.
# IMPORTANT
### maxDepth of 5 on a 3x3 board improves performace by 4X (from just over 4 seconds to less than 1 second) and still plays optimally.

In [216]:
class Minimax(Algorithm):
    def __init__(self, boardGame):
        super().__init__(boardGame)

    # function to find the best move
    def bestMove(self, boardGame, letter):
        boardState = boardGame.getBoardState()
        bestScore = -1000
        bestMove = 0
        for key in boardState.keys():
            if boardState[key] == ' ':
                boardState[key] = letter
                score = self.minimax(boardState, 0, False, letter)
                boardState[key] = ' '
                if score > bestScore:
                    bestScore = score
                    bestMove = key
        return bestMove

    # minimax function
    def minimax(self, boardState, depth, isMaximizing, letter, maxDepth=2):
        gameLogic = TicTacToeGameLogic(self.boardGame)
        if letter == 'X':
            opponentLetter = 'O'
        else:
            opponentLetter = 'X'
        if(gameLogic.chkMarkForWin(letter)):
            return 1
        elif(gameLogic.chkMarkForWin(opponentLetter)):
            return -1
        elif(gameLogic.chkForDraw()):
            return 0
        elif depth >= maxDepth:
            return 0
            
        if isMaximizing:
            bestScore = -1000
            for key in boardState.keys():
                if boardState[key] == ' ':
                    boardState[key] = letter
                    score = self.minimax(boardState, depth + 1, False, letter)
                    boardState[key] = ' '
                    if score > bestScore:
                        bestScore = score
            return bestScore
        else:
            bestScore = 1000
            for key in boardState.keys():
                if boardState[key] == ' ':
                    boardState[key] = opponentLetter
                    score = self.minimax(boardState, depth + 1, True, letter)
                    boardState[key] = ' '
                    if score < bestScore:
                        bestScore = score
            return bestScore

In [217]:
# create a user input algorithm class
class UserInput(Algorithm):
    def __init__(self, boardGame):
        super().__init__(boardGame)

    # function to get the best move
    def bestMove(self, boardGame, letter):
        position = int(input("Please enter a position: "))
        return position

## Game
#### Entry Point

In [218]:
def selectAlgorithm(boardGame):
    # list the algorithms
    print("1. Minimax")
    print("2. Reinforcement Learning")
    print("3. User Input")
    # ask the user to choose an algorithm
    algorithm_choice = int(input("Please choose an algorithm: "))
    while algorithm_choice < 1 or algorithm_choice > 3:
        algorithm_choice = int(input("Please choose an algorithm: "))
    # create the algorithm object based on the user's choice
    if algorithm_choice == 1:
        algorithm = Minimax(boardGame=boardGame)
    elif algorithm_choice == 2:
        algorithm = ReinforcementLearning(boardGame=boardGame)
    else:
        algorithm = UserInput(boardGame=boardGame)
    return algorithm

def gameLoop():
    # ask for board dimensions
    # must be between 3 and 5
    # boardDimensions = int(input("Please enter the board dimensions: "))
    # while boardDimensions < 3 or boardDimensions > 5:
    #     boardDimensions = int(input("Please enter the board dimensions: "))
    boardDimensions = 4
    # Create a tictactoe board with the dimensions
    boardGame = TicTacToeBoard(boardDimensions)
    # Create a tictactoe game logic
    gameLogic = TicTacToeGameLogic(boardGame)
    # select the algorithm for playerOne
    # algorithm = selectAlgorithm(boardGame)
    algorithm = Minimax(boardGame)
    # Create playerOne
    playerOne = Player('X', algorithm)
    # select the algorithm for playerTwo
    algorithm = selectAlgorithm(boardGame)
    # Create playerTwo
    playerTwo = Player('O', algorithm)
    # print the board
    boardGame.printBoard()
    #start the game
    while not gameLogic.chkForWin() and not gameLogic.chkForDraw():
        # ask playerOne to choose a position using the algorithm
        position = playerOne.algorithm.bestMove(boardGame, playerOne.letter)
        # insert letter into the position
        gameLogic.insertLetter(playerOne.letter, position)
        # check for win or draw
        if gameLogic.chkForWin():
            print("Player", playerOne.letter, "wins!")
            break
        elif gameLogic.chkForDraw():
            print("It's a draw!")
            break
        # ask playerTwo to choose a position
        position = playerTwo.algorithm.bestMove(boardGame, playerTwo.letter)
        # insert letter into the position
        gameLogic.insertLetter(playerTwo.letter, position)
        # check for win or draw
        if gameLogic.chkForWin():
            print("Player", playerTwo.letter, "wins!")
            break
        elif gameLogic.chkForDraw():
            print("It's a draw!")
            break

# time how long gameLoop takes to run
import time
start = time.time()
gameLoop()
end = time.time()
print("Time taken: ", end - start)


1. Minimax
2. Reinforcement Learning
3. User Input
 | | | 
-------
 | | | 
-------
 | | | 
-------
 | | | 




KeyboardInterrupt: 